In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf
import glob
import numpy as np
from PIL import Image
from sklearn.metrics import pairwise_distances, roc_auc_score
from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import tqdm
import sys
sys.path.append("../")
from inception_resnet_v1 import inference

In [2]:
def prewhiten(x):
    """
    A helper function to whiten an image, or a batch of images.
    Args:
        x: An image or batch of images.
    """
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        print(x.ndim)
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
#     std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    """
    Normalizes an embedding to have unit length in the l2 metric.
    Args:
        x: A batch of numpy embeddings
    """
    output = x / np.sqrt(np.maximum(np.sum(np.square(x),
                                           axis=axis,
                                           keepdims=True),
                                    epsilon))
    return output

In [3]:
class PersonImages:
    def __init__(self, person_name):
        self.clean_folder = ""
        self.clean_images = []
        self.adversarial_images = []
        self.orig_mean = None
        self.orig_std = None
        self.orig_paths = []
        self.person_name = person_name
    
    def _load_one_facenet(self, path, resize_size=None, prewhiten_img=True):
        img = Image.open(path).convert("RGB")
    
            
        if resize_size:
            img = img.resize((resize_size, resize_size))
            
        img = (np.array(img)).astype(np.float32)
        
        if prewhiten_img:
            img = prewhiten(img)
            
        return img
        
    def _load_folder_for_facenet(self, folder, resize_size=None):
        paths_list = glob.glob(os.path.join(folder, "*"))
        final_imgs = []
        for img_path in paths_list:
            final_imgs.append(
                self._load_one_facenet(
                    img_path, prewhiten_img=False, resize_size=resize_size))
        
        final_imgs = np.array(final_imgs)
        mean, std = np.mean(final_imgs), np.std(final_imgs)
        final_imgs = prewhiten(np.array(final_imgs))
        return final_imgs, mean, std, paths_list
        
    def load_clean_from_folder(self, clean_folder, resize_size=160):
        self.clean_folder = clean_folder
        self.clean_images, self.orig_mean, self.orig_std, self.orig_paths = self._load_folder_for_facenet(
            clean_folder, resize_size=resize_size)
    
    def _undo_preprocess(self, images):
        restored_images = images.copy()
        restored_images  *= self.orig_std
        restored_images += self.orig_mean
        restored_images = np.clip(restored_images, 0.0, 255.0)
        return np.uint8(restored_images)
    
    def _compute_embeddings(self, model, images):
        return model.predict(np.array(images), batch_size=len(images))
    
    def get_clean_for_display(self):
        return self._undo_preprocess(self.clean_images)
    
    def compute_clean_embeddings_with_model(self, model):
        self.clean_embeddings = self._compute_embeddings(model, self.clean_images)

In [4]:
class MyModel:
    def __init__(self, model_path, inputs=None, sess=None):
        if inputs is None:
            self.model_inputs = tf.placeholder(tf.float32, shape=(None, 160, 160, 3))
        else:
            self.model_inputs = inputs
        
        vars_before = tf.global_variables()
        self.net, self.endpoints = inference(
            self.model_inputs, keep_probability=1.0, bottleneck_layer_size=512, phase_train=False)
        vars_after = tf.global_variables()
        
        model_name = dataset_to_model_name["vggface2"]
        saver = tf.train.Saver(list(set(vars_after) - set(vars_before)))
        
        if sess is None:
            self.sess = tf.Session()
        else:
            self.sess = sess
            
        saver.restore(self.sess, model_path)
    
    def predict(self, inputs, batch_size):
        return self.sess.run(self.net, feed_dict={self.model_inputs: inputs})
    
    def __del__(self):
        self.sess.close()

dataset_to_model_name = {
    "casia-webface": "20180408-102900",
    "vggface2": "20180402-114759"
}

dataset_to_ckpt_number = {
    "casia-webface": "90",
    "vggface2": "275"
}

def build_model(dataset_name, inputs=None, sess=None):
    model_name = dataset_to_model_name[dataset_name]
    model = MyModel(
        os.path.join(
            "/home/ivan/facenet/models",
            model_name,
            "model-{model_name}.ckpt-{ckpt_num}".format(
                model_name=dataset_to_model_name[dataset_name],
                ckpt_num=dataset_to_ckpt_number[dataset_name]
            )),
        inputs, 
        sess
    )
    return model

In [5]:
def path_for_id_clean(identity):
    if identity == "n000958":
        return "/data/vggface/test_perturbed_sampled/{id}/community_naive_mean/n000029/epsilon_0.0/png".format(id=identity)
    else:
        return "/data/vggface/test_perturbed_sampled/{id}/community_naive_mean/n000958/epsilon_0.0/png".format(id=identity)
    

In [6]:
def build_person(person_name, path_for_id_fn, model=None):
    person_a = PersonImages(person_name)
    person_a.load_clean_from_folder(path_for_id_fn(person_a.person_name))
    if model is not None:
        print("Computing embeddings for", person_name)
        person_a.compute_clean_embeddings_with_model(model)
    return person_a

In [7]:
vggface2_model = build_model("vggface2")
people = [
    build_person(person_name, path_for_id_clean, vggface2_model) \
    for person_name in os.listdir("/data/vggface/test_perturbed_sampled")
]
del vggface2_model

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Computing embeddings for n001781
Computing embeddings for n009232
Computing embeddings for n000958
Computing embeddings for n003356
Computing embeddings for n008655
Computing embeddings for n008613
Computing embeddings for n004658
Computing embeddings for n001683
Computing embeddings for n002647
Computing embeddings for n009288
Computing embeddings for n005427
Computing embeddings for n002763
Computing embeddings for n002503
Computing embeddings for n003215
Computing embeddings for n005359
Computing embeddings for n005303
Computing embeddings for n007548
Computing embeddings for n000029
Computing embeddings for n009114


In [8]:
def build_vggface_model(inputs, sess):
    return build_model("vggface2", inputs, sess)

def build_casiawebface_model(inputs, sess):
    return build_model("casia-webface", inputs, sess)

    
def attack_batch(
    build_model_fn, 
    input_images,
    target_vectors,
    learning_rate,
    epsilon,
    max_iters
):
    input_images = np.array(input_images)
    batch_size, orig_h, orig_w, orig_c = input_images.shape
    
    with tf.Graph().as_default():
        with tf.Session() as sess:
            targets_var = tf.get_variable(
                name="targets",
                shape=target_vectors.shape,
            )
            sess.run(tf.assign(targets_var, target_vectors))
            
            noisy_images_var = tf.get_variable(
                name="noisy_images",
                shape=input_images.shape
            )
            sess.run(tf.assign(noisy_images_var, input_images + tf.random.normal(input_images.shape, 0.0, 0.1)))

#             noise_var = tf.get_variable(
#                 name="adversarial_noise",
#                 shape=input_images.shape,
#                 initializer=tf.initializers.truncated_normal(
#                     mean=np.mean(input_images),
#                     std=np.std(input_images)
#                 )
#             )
            
                        
            randomized_images_plus_noise = tf.image.random_brightness(
                    noisy_images_var, 0.5)

            randomized_images_plus_noise = tf.image.random_crop(
                randomized_images_plus_noise, 
                [batch_size, orig_h - 10, orig_w - 10, 3]
            )
            
            randomized_images_plus_noise = tf.image.resize_images(
                randomized_images_plus_noise, [orig_h, orig_w])

            randomized_images_plus_noise += tf.random.normal(
                randomized_images_plus_noise.shape, 0.0, 0.75)
            
            model = build_model_fn(
                inputs=randomized_images_plus_noise, 
                sess=sess
            )
            
            model_outputs = tf.nn.l2_normalize(model.net, axis=0)
            targets = tf.nn.l2_normalize(targets_var, axis=0)
            loss = tf.reduce_sum(tf.multiply(model_outputs, targets))
            losses = []
            for i in range(max_iters):
                grads = tf.gradients(loss, [noisy_images_var])[0]
                assert grads is not None
                losses.append(sess.run(loss))
                sess.run([
                    tf.assign(
                        noisy_images_var,
                        noisy_images_var - learning_rate * tf.sign(grads)
                    ),
                    tf.assign(
                        noisy_images_var, 
                        tf.clip_by_value(
                            noisy_images_var, 
                            input_images - epsilon,
                            input_images + epsilon
                        ))
                ])
            return sess.run(noisy_images_var), losses

In [9]:
def generate_decoys(
    people_list,
    attack_strategy,
    model_to_attack,
    learning_rate,
    epsilon,
    max_iters
):
    if model_to_attack == "vggface2":
        model_build_fn = build_vggface_model
    elif model_to_attack == "casia-webface":
        model_build_fn = build_casiawebface_model
        
    for person in people_list:
        num, height, width, channels = person.clean_images.shape
        person.adversarial_images = np.zeros((2*(len(people_list) - 1), height, width, channels))
        indx = 0
        for other_person in people_list:
            if person.person_name == other_person.person_name:
                continue
            target_vector = np.mean(other_person.clean_embeddings, axis=0)
            images_to_make_adversarial = person.clean_images[indx : indx + 2] 
            targets_for_images = np.array([target_vector for _ in range(len(images_to_make_adversarial))])
            person.adversarial_images[indx : indx + 2], losses = attack_batch(
                model_build_fn, 
                images_to_make_adversarial,
                targets_for_images,
                learning_rate,
                epsilon,
                max_iters
            )
            print("Done generating decoys by", person.person_name, "for", other_person.person_name)
            
            fig, ax = fig1, ax1 = plt.subplots()
            ax.plot(range(len(losses)), losses)
            
            save_dest = os.path.join(
                "/data/vggface/test_perturbed_sampled",
                person.person_name,
                "{attack_strategy}_{model}".format(attack_strategy=attack_strategy, model=model_to_attack),
                other_person.person_name
            )
            
            save_path = os.path.join(save_dest, "epsilon_{}".format(epsilon), "png")
            os.makedirs(save_path, exist_ok=True)
            existing_files = os.listdir(save_path)

            # Clean up folder if need be
            if len(existing_files) > 0:
                for f in existing_files:
                    os.remove(os.path.join(save_path, f))
            
            for i in range(indx, indx + 2):
                orig_name = person.orig_paths[i].split("/")[-1]
                
                full_save_path = os.path.join(save_path, orig_name)
                print("Saving to", save_path)
                Image.fromarray(
                    person._undo_preprocess(person.adversarial_images[i])
                ).save(full_save_path)

            indx += 2

In [ ]:
generate_decoys(
    people,
    "mean",
    "vggface2",
    0.1,
    0.1,
    2
)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n001781 for n009232
Saving to /data/vggface/test_perturbed_sampled/n001781/mean_vggface2/n009232/epsilon_0.1/png/22.png
Saving to /data/vggface/test_perturbed_sampled/n001781/mean_vggface2/n009232/epsilon_0.1/png/27.png
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n001781 for n000958
Saving to /data/vggface/test_perturbed_sampled/n001781/mean_vggface2/n000958/epsilon_0.1/png/25.png
Saving to /data/vggface/test_perturbed_sampled/n001781/mean_vggface2/n000958/epsilon_0.1/png/17.png
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating 

INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n009232 for n003356
Saving to /data/vggface/test_perturbed_sampled/n009232/mean_vggface2/n003356/epsilon_0.1/png/3.png
Saving to /data/vggface/test_perturbed_sampled/n009232/mean_vggface2/n003356/epsilon_0.1/png/15.png


/home/ivan/anaconda3/envs/facenet-gpu-112/lib/python3.5/site-packages/matplotlib/pyplot.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n009232 for n008655
Saving to /data/vggface/test_perturbed_sampled/n009232/mean_vggface2/n008655/epsilon_0.1/png/16.png
Saving to /data/vggface/test_perturbed_sampled/n009232/mean_vggface2/n008655/epsilon_0.1/png/42.png
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n009232 for n008613
Saving to /data/vggface/test_perturbed_sampled/n009232/mean_vggface2/n008613/epsilon_0.1/png/35.png
Saving to /data/vggface/test_perturbed_sampled/n009232/mean_vggface2/n008613/epsilon_0.1/png/41.png
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating 

INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n000958 for n004658
Saving to /data/vggface/test_perturbed_sampled/n000958/mean_vggface2/n004658/epsilon_0.1/png/28.png
Saving to /data/vggface/test_perturbed_sampled/n000958/mean_vggface2/n004658/epsilon_0.1/png/32.png
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n000958 for n001683
Saving to /data/vggface/test_perturbed_sampled/n000958/mean_vggface2/n001683/epsilon_0.1/png/33.png
Saving to /data/vggface/test_perturbed_sampled/n000958/mean_vggface2/n001683/epsilon_0.1/png/45.png
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n000958 for n002647
Saving to /data/vgg

Done generating decoys by n003356 for n002647
Saving to /data/vggface/test_perturbed_sampled/n003356/mean_vggface2/n002647/epsilon_0.1/png/40.png
Saving to /data/vggface/test_perturbed_sampled/n003356/mean_vggface2/n002647/epsilon_0.1/png/14.png
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n003356 for n009288
Saving to /data/vggface/test_perturbed_sampled/n003356/mean_vggface2/n009288/epsilon_0.1/png/23.png
Saving to /data/vggface/test_perturbed_sampled/n003356/mean_vggface2/n009288/epsilon_0.1/png/11.png
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n003356 for n005427
Saving to /data/vggface/test_perturbed_sampled/n003356/mean_vggface2/n005427/epsilon_0.1/png/1.png
Saving to /data/vggface/test_pertur

INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n008655 for n002763
Saving to /data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n002763/epsilon_0.1/png/2.png
Saving to /data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n002763/epsilon_0.1/png/5.png
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n008655 for n002503
Saving to /data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n002503/epsilon_0.1/png/30.png
Saving to /data/vggface/test_perturbed_sampled/n008655/mean_vggface2/n002503/epsilon_0.1/png/10.png
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Done generating decoys by n008655 for n003215
Saving to /data/vggfa